In [50]:
import os
import jieba
import re
from math import log

In [51]:
# 保留中文内容
# 爬虫下来的文档中会有冗余信息
def find_chinese(file):
    pattern = re.compile(r'[^\u4e00-\u9fa5]')
    chinese = re.sub(pattern, '', file)
    return chinese

In [52]:
def readFIles(files: list(list())):
    # 读取data文件夹下的所有文件
    for file in os.listdir('data'):
        text = open(f'data/{file}', 'r').read()
        text = find_chinese(text)
        
        # 根据文件名进行分类
        if file.startswith('党政办公室'):
            files[0].append(text)
        elif file.startswith('教务部'):
            files[1].append(text)
        elif file.startswith('招生办公室'):
            files[2].append(text)
        elif file.startswith('研究生院'):
            files[3].append(text)
        elif file.startswith('科学技术部'):
            files[4].append(text)

In [53]:
# 利用jieba进行分析
def cutWords(files: list(list()), terms: list(list()), term_list: set()):
    for index, file in enumerate(files):
        for text in file:
            words = list(jieba.cut(text, cut_all=False))
            terms[index].append(words)
            term_list.update(set(words))

In [54]:
# 计算MI值
def cal_MI(N11, N01, N10, N00):
    N = N11+N01+N10+N00
    sum = 0
    sum += 0 if N11 == 0 else N11/N*log(N*(N11)/(N11+N10)/(N11+N01), 2)
    sum += 0 if N01 == 0 else N01/N*log(N*N01/(N01+N00)/(N11+N01), 2)
    sum += 0 if N10 == 0 else N10/N*log(N*N10/(N10+N11)/(N10+N00), 2)
    sum += 0 if N00 == 0 else N00/N*log(N*N00/(N00+N01)/(N00+N10), 2)
    return sum

In [55]:
# 计算X^2值
def cal_X2(N11, N01, N10, N00):
    sum = (N11+N10+N01+N00)*((N11*N00-N10*N01)**2)/(N11+N01+1)/(N10+N00+1)/(N11+N10+1)/(N01+N00+1)
    return sum

In [56]:
# 重建小顶堆
def sift(li,low,high):
    i = low
    #找孩子 左孩子
    j = 2 * i +1
    tmp = li[low] #把堆顶存起来
    while  j <= high: #只要j位置有数
        #右孩子要右，右孩子比较大 并且右孩子大于左孩子     右孩子不越界 j+1 <=hight 
        if j+1 <=high  and li[j+1][1] < li[j][1]:
        #j 指向右孩子  这个指的是两个孩子更大的数
            j = j + 1
        if li[j][1] < tmp[1]:  #目前要是,tmp大就放过去,还是j大放上面去
            li[i] = li[j]
            i = j       #可以交换了    现在j等于新的i     i等于原来的j
            j = 2 * i + 1
        else :  #tmp 更大,把tmp放到i的位置上  假如堆点是6
            li[i] = tmp  # 6放到原来8的
            break
    else: #就是2没法和别的比了,就放到该去的地方去
        li[i] = tmp  #把tmp放到叶子节点上去

#最后非叶子节点
#n的最后一个下标是n-1,找他的父亲就孩子,(n-1-1)/2
#孩子下标是 n
#孩子找父亲 (n-1)/2 里面的n的最后一个下标是n-1 所有(n-1-1)/2 -> (n-2)/2

def topk(li,k):
    # 前K元素建立堆
    heap = li[0:k]
    for i in range((k-2)//2,-1,-1):
        sift(heap,i,k-1)

    #1.建堆
    for i in range(k,len(li)):
        if li[i][1] > heap[0][1]:
            heap[0] = li[i]
            sift(heap,0,k-1)

    #2. 堆排序
    for i in range(k-1,-1,-1):
        heap[0], heap[i] = heap[i], heap[0]
        sift(heap,0,i -1)
    return heap

In [57]:
if __name__ == "__main__":
    term_list = set()
    # 0 --> 党政办公室
    # 1 --> 教务部
    # 2 --> 招生办公室
    # 3 --> 研究生院
    # 4 --> 科学技术部
    files = [[] for i in range(5)]
    terms = [[] for i in range(5)]

    # 依次打开data文件夹下的文件
    readFIles(files)

    # 进行中文分词，并生成词典
    cutWords(files, terms, term_list)

    # 计算每个单词的互信息
    MI = [{} for i in range(5)]
    X2 = [{} for i in range(5)]
    for word in term_list:
        cat_t, cat_f = [[] for i in range(5)], [[] for i in range(5)]
        
        # 统计每一个category中的出现情况
        # cat_t[i] 表示第i个category中出现的word的doc的个数
        # cat_f[i] 表示第i个category中不出现的word的doc的个数
        for index in range(5):
            cat_t[index] = list(word in words for words in terms[index]).count(True)
            cat_f[index] = list(word in words for words in terms[index]).count(False)
        
        # 计算混淆矩阵
        for index in range(5):
            N11 = cat_t[index]
            N01 = cat_f[index]
            N10 = sum(cat_t)-N11
            N00 = sum(cat_f)- N01
            # 计算MI值
            score1 = cal_MI(N11, N01, N10, N00)
            socre2 = cal_X2(N11, N01, N10, N00)
            
            # 将计算得到的MI值插入对应类别中
            MI[index][word] = score1
            X2[index][word] = socre2
    
    # 计算每个类别的TopK
    print('根据MI计算TopK:')
    print(f'党政办公室MI值Top15为:\n{topk(list(MI[0].items()), 15)}\n')
    print(f'教务部MI值Top15为:\n{topk(list(MI[1].items()), 15)}\n')
    print(f'招生办公室MI值Top15为:\n{topk(list(MI[2].items()), 15)}\n')
    print(f'研究生院MI值Top15为:\n{topk(list(MI[3].items()), 15)}\n')
    print(f'科学技术部MI值Top15为:\n{topk(list(MI[4].items()), 15)}\n')
    
    print('==============================================')
    
    print('根据X2计算TopK:')
    print(f'党政办公室X2值Top15为:\n{topk(list(X2[0].items()), 15)}\n')
    print(f'教务部X2值Top15为:\n{topk(list(X2[1].items()), 15)}\n')
    print(f'招生办公室X2值Top15为:\n{topk(list(X2[2].items()), 15)}\n')    
    print(f'研究生院X2值Top15为:\n{topk(list(X2[3].items()), 15)}\n')
    print(f'科学技术部X2值Top15为:\n{topk(list(X2[4].items()), 15)}\n')

根据MI计算TopK:
党政办公室MI值Top15为:
[('党政', 0.6252387577646294), ('办公室', 0.24140916176572477), ('定于', 0.15547585849043016), ('全校', 0.13112582951135474), ('党委', 0.12068674573587532), ('培养', 0.10508346537777384), ('总支', 0.09790657010494239), ('基层', 0.09790657010494239), ('筹', 0.08083781598886139), ('南山区', 0.08083781598886139), ('保密工作', 0.08083781598886139), ('博物馆', 0.08083781598886139), ('一名', 0.08083781598886139), ('申请', 0.07568595749149351), ('研究', 0.07290559532005605)]

教务部MI值Top15为:
[('教务部', 0.533831243936691), ('教学', 0.17273374756613916), ('课程', 0.11989126922622834), ('办公室', 0.10267485000934576), ('党政', 0.08132896032627286), ('也', 0.08132896032627286), ('门', 0.08083781598886139), ('招生', 0.08065984429186183), ('研究生院', 0.07849356477829864), ('广东省教育厅', 0.07598475081979274), ('科学技术部', 0.07568595749149351), ('四六级', 0.0640875518754165), ('思政', 0.0640875518754165), ('教高', 0.0640875518754165), ('上网', 0.0640875518754165)]

招生办公室MI值Top15为:
[('中学', 0.4899239356630936), ('招生', 0.44643934467101554), ('走